In [ ]:
class Config: 
    """
    In this competition the matrix size is from 2x2 to 30x30
    and maximum 5-6 examples of solutions 
    """
    
    
    train = {'inputs': '/kaggle/input/arc-prize-2024/arc-agi_training_challenges.json',
             'outputs': '/kaggle/input/arc-prize-2024/arc-agi_training_solutions.json'}
    validation = {'inputs': '/kaggle/input/arc-prize-2024/arc-agi_evaluation_challenges.json', 
                  'outputs': '/kaggle/input/arc-prize-2024/arc-agi_evaluation_solutions.json'}
    test = {'inputs': '/kaggle/input/arc-prize-2024/arc-agi_test_challenges.json'}
    
cfg = Config() 

In [ ]:
import numpy as np 
import json 

from tensorflow.data import Dataset 
import tensorflow as tf 

from tensorflow.keras import layers 
from tensorflow.keras.optimizers import AdamW 

# Data Preparation 

In [ ]:
def prepare_inputs(path): 
    with open(path, mode='r') as file: 
        return json.load(file)
    
def prepare_outputs(path): 
    with open(path, mode='r') as file: 
        return json.load(file)

def prepare1(data_path_item): 
    inp = 'inputs'
    out = 'outputs'
    r = dict() 
    
    if inp in data_path_item: 
        inputs = data_path_item[inp]
        
        r['inputs'] = prepare_inputs(inputs)
        
    if out in data_path_item: 
        outputs = data_path_item[out]
        
        r['outputs'] = prepare_outputs(outputs)
        
    return r 

train = prepare1(cfg.train)
validation = prepare1(cfg.validation)
# test = prepare1(cfg.test)

In [ ]:
ex = list(train['inputs'].values())[10]

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2 

def size_to_30(i):
    arr = np.array(i['input'])
    # Convert arr to cv2 image (assuming arr is in a format compatible with OpenCV)
    img = arr.astype(np.uint8)
    # Resize the image to 30x30 using cubic interpolation
    img_resized = cv2.resize(img, dsize=(100, 100), interpolation=cv2.INTER_NEAREST)
    # Convert the resized image back to a NumPy array
    return img_resized


# Assuming ex['test'] contains the original arrays in 'input' field
original = [np.array(i['input']) for i in ex['test']]
resized = [size_to_30(i) for i in ex['test']]

# Visualize the original image
plt.imshow(original[0], cmap='gray')
# plt.set_title('Original')
# plt.axis('off')
plt.show() 

# Visualize the resized image
plt.imshow(resized[0], cmap='gray')
# plt.set_title('Resized (30x30)')
# plt.axis('off')

# Adjust layout
plt.tight_layout()
plt.show()
